In [56]:
import os
import numpy
from stl import mesh
from mpl_toolkits import mplot3d
from matplotlib import pyplot

### Price Calcs

- Cost per ml: \$0.149 to \$0.199
- Material Factor: 1.5x to 3x, avg of 2x
- Labor Factor: a base price or a multiplier

### Factors

- Fractal was \$0.30/$cm^3$
- Shapeways is \$2.35/$cm^3$
- Protolabs is \$14.20/$cm^3$

### Data

#### Volume Data

| part                              | volume pre-process | volume post-process | volume ratio |
|-----------------------------------|--------------------|---------------------|--------------|
| std10mm_Bar Type - Housing Bottom | 12.47              | 26.57               | 2.13         |
| std10mm_Bar Type - Housing Top    | 10.01              | 23.91               | 2.39         |
| eng10mm_Bar Type - Rubber Top     | 7.42               | 14.23               | 1.92         |
| eng10mm_Bar Type - Rubber Bottom  | 7.38               | 21.94               | 2.97         |
|                                   |                    |                     |              |
| std20mm_Prototype - Sleeve Half   | 2.03               | 4.41                | 2.17         |
| std20mm_Prototype - Release Rod   | 1.54               | 2.38                | 1.55         |
| std20mm_Prototype - Body Half     | 2.44               | 5.9                 | 2.42         |
| eng20mm_Prototype - Rubber Half   | 2.23               | 3.6                 | 1.61         |

#### Volume Ratios

| Volume Ratio |      |
|--------------|------|
| min:         | 1.55 |
| max:         | 2.97 |
| mean:        | 2.15 |
| std:         | 0.46 |

#### 3d Pricer Data

| material rate | material factor | labor factor | 3dpricer  |
|---------------|-----------------|--------------|-----------|
| 0.149         | 2               | 10           | \$37.16   |
| 0.149         | 2               | 10           | \$29.83   |
| 0.199         | 2               | 10           | \$29.53   |
| 0.199         | 2               | 10           | \$29.37   |
| 0.149         | 2               | 10           | \$6.05    |
| 0.149         | 2               | 10           | \$4.59    |
| 0.149         | 2               | 10           | \$7.27    |
| 0.199         | 2               | 10           | \$8.88    |

#### Manual Pricer Data

| manual pricer | pricer ratio |
|---------------|--------------|
| \$39.59        | 0.94         |
| \$35.63        | 0.84         |
| \$28.32        | 1.04         |
| \$43.66        | 0.67         |
| \$6.57         | 0.92         |
| \$3.55         | 1.29         |
| \$8.79         | 0.83         |
| \$7.16         | 1.24         |

#### Pricer Ratios

| Pricer Ratio |      |
|--------------|------|
| min:         | 0.67 |
| max:         | 1.29 |
| mean:        | 0.97 |
| std:         | 0.21 |

#### Notes:
- Pricer ratios are only affected by the material factor.
- The labor factor is in both prices, so it doesn't affect the ratio (it drops out)

In [57]:
materials = {'pla': .01,
             'std': .149,
             'eng': .199,
             'shapeways': 2.35,
             'protolabs': 14.20}

resolution = {'lo': 1.0,
              'md': 1.5,
              'hi': 2.0}

conversion = {'in': 16387.064,
              'mm': 1}

rates = {'material': 5,
         'labor': 8,
         'materials': 5,
         'order_min': 25,
         'part_min': 15,
         'editing': 75}

finishing = {'none': 1.0,
             'basic': 1.5,
             'sanded': 2.0,
             'primed': 3.0,
             'painted': 4.0}

In [58]:
def get_stl_image(meta):
    file = meta['path']
    name = meta['name']
    price = meta['price']
    qty = meta['qty']
    # Create a new plot
    figure = pyplot.figure()
    axes = mplot3d.Axes3D(figure)
    axes.set_title("{0:.0f} x {1:s} - ${2:.2f}".format(qty,name,price), fontsize=14, fontweight='bold')

    # Load the STL files and add the vectors to the plot
    your_mesh = mesh.Mesh.from_file(file)
    axes.add_collection3d(mplot3d.art3d.Poly3DCollection(your_mesh.vectors))

    # Auto scale to the mesh size
    scale = your_mesh.points.flatten(-1)
    axes.auto_scale_xyz(scale, scale, scale)

    # Show the plot to the screen
    pyplot.axis('off')
    image = pyplot.show()
    return image

# example
# stdmd20mm_name
# 0123456789

def get_part_files(part_dir):
    part_dir = 'Parts/' + part_dir
    part_files = {}
    for file in os.listdir(part_dir):
        if file.endswith(".stl"):            
            meta_data = {}
            material = file[:3]
            resolution = file[3:5]
            qty = int(file[5])
            edit = int(file[6])
            unit = file[7:9]
            name = file[10:]
            meta_data["name"] = name
            meta_data["path"] = part_dir + "/" + file
            meta_data["material"] = material
            meta_data["resolution"] = resolution
            meta_data["qty"] = qty
            meta_data["edit"] = edit
            meta_data["unit"] = unit
            part_files[file] = meta_data
    return part_files

def get_rate_pricing(part_dir):
    total = 0
    part_total = {}
    part_pricing = {}
    parts = get_part_files(part_dir)
    for part in parts:
        # STL data
        meta = parts[part]
        part_mesh = mesh.Mesh.from_file(meta['path'])
        volume, cog, inertia = part_mesh.get_mass_properties()
        volume = volume*.001*conversion[meta['unit']]
        meta['volume'] = volume
        # Price Calculation
        material_cost = materials[meta['material']]*resolution[meta['resolution']]*volume*rates['labor']*rates['material']*meta['qty']
        editing_cost = meta['edit']*rates['editing']
        part_price = material_cost + editing_cost
        # Save Price & Total
        total += part_price
        meta['price'] = part_price
        part_pricing[part] = meta
    part_total['price'] = total
    part_total['name'] = "Total"
    part_total['order'] = part_dir
    part_pricing['total'] = part_total
    return part_pricing

def get_base_pricing(part_dir):
    parts = get_part_files(part_dir)
    total = 0
    count = 0
    part_total = {}
    part_pricing = {}
    for part in parts:
        meta = parts[part]
        qty = meta['qty']
        count += qty
    part_base = rates['part_min'] + rates['order_min']/qty
    for part in parts:
        # STL data
        meta = parts[part]
        part_mesh = mesh.Mesh.from_file(meta['path'])
        volume, cog, inertia = part_mesh.get_mass_properties()
#         area = MeshFacesArea(part_mesh)
#         meta['area'] = area
        volume = volume*.001*conversion[meta['unit']]
        meta['volume'] = volume
        # Price Calculation
        material_cost = materials[meta['material']]*resolution[meta['resolution']]*volume*rates['materials']*meta['qty']
        editing_cost = meta['edit']*rates['editing']
        part_price = material_cost + editing_cost + part_base
        # Save Price & Total
        total += part_price
        meta['price'] = part_price
        part_pricing[part] = meta
    part_total['price'] = total
    part_total['name'] = "Total"
    part_total['order'] = part_dir
    part_pricing['total'] = part_total
    return part_pricing


In [59]:
def print_pricing(part_pricing, print_volume, print_image):
    print(part_pricing['total']['order'])
    for part in part_pricing:
        meta = part_pricing[part]
        if part != 'total':
            print("{0:.0f} x ".format(meta['qty']), end='')
            if print_image == 1:
                get_stl_image(meta)
            if print_volume == 1:
                volume = meta['volume']
                unit = meta['unit']
                print(" {0:.2f}{1:s}^3 ".format(volume,unit), end='')
#                 print(meta['area'])
        print("{0:s} - ${1:.2f}".format(meta['name'],meta['price']))
    return None

def print_figures(part_pricing):
    print('Figures:')
    for part in part_pricing:
        meta = part_pricing[part]
        if part != 'total':
            get_stl_image(meta)
        else:
            print("{0:s} - ${1:.2f}".format(meta['name'],meta['price']))
    return None

### Test

In [60]:
# print("Rate")
# print_pricing(get_rate_pricing("project_WU"),0,0)
# print("Base")
# print_pricing(get_base_pricing("project_WU"),0,0)
# print("\n")
# print("Rate")
# print_pricing(get_rate_pricing("project_DM"),0,0)
# print("Base")
# print_pricing(get_base_pricing("project_DM"),0,0)

In [61]:
# print_figures(get_base_pricing("project_DM"))

In [62]:
# print_pricing(get_base_pricing("Clear Prototype"),0,0)
# print_pricing(get_base_pricing("Color Prototype"),0,0)

In [63]:
# print_pricing(get_base_pricing("MCA"),0,0)

In [64]:
# print_pricing(get_base_pricing("Band"),0,0)

In [65]:
# print_pricing(get_base_pricing("DM Small Prototype"),0,0)
# print_pricing(get_base_pricing("DM Big Prototype"),0,0)

In [66]:
# print_pricing(get_base_pricing("SWS Proto"),0,0)

In [67]:
# print_pricing(get_rate_pricing("Bottle Lid"),0,0)

In [68]:
# print_pricing(get_rate_pricing("Packaging Samples"),0,0)

# print_pricing(get_base_pricing("Packaging Samples"),0,0)

print_pricing(get_base_pricing("Meeples"),0,0)

Your mesh is not closed, the mass methods will not function
            correctly on this mesh.  For more info:
            https://github.com/WoLpH/numpy-stl/issues/69
Your mesh is not closed, the mass methods will not function
            correctly on this mesh.  For more info:
            https://github.com/WoLpH/numpy-stl/issues/69
Your mesh is not closed, the mass methods will not function
            correctly on this mesh.  For more info:
            https://github.com/WoLpH/numpy-stl/issues/69
Your mesh is not closed, the mass methods will not function
            correctly on this mesh.  For more info:
            https://github.com/WoLpH/numpy-stl/issues/69


Meeples
1 x P4_Meeple.stl - $41.03
1 x P1_Meeple.stl - $42.04
1 x P3_Meeple.stl - $41.25
1 x P5_Meeple.stl - $41.93
1 x P2_Meeple.stl - $41.67
Total - $207.91


In [69]:
print_pricing(get_base_pricing("jeff"),1,1)

UnboundLocalError: local variable 'qty' referenced before assignment